# Michaelis-Menten kinetics dynamics

### The Michaelis-Menten kinetics is given by the following system of (coupled) ordinary differential equations (ODEs):

$$
\dot{x}^{(\nu)}_i = -x^{(\nu)}_i + \sum_j w_{i,j} \cdot \frac{x^{(\nu)}_j}{1 + x^{(\nu)}_j}, \quad i, j \in \{1, \dots, N\}
$$

In [85]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint  # Import the odeint function from scipy for solving ODEs
from scipy.integrate import solve_ivp  # Import the solve_ivp function from scipy for solving ODEs
import random

In [86]:
# Set up the system of ODEs
def ode_system(x, t, W):  # t must be passed even if not used
    N = len(x)  # Get the number of elements in the state vector
    derivative_vector = np.zeros(N)  # Initialize the derivative vector (set zeros as default)
    for i in range(N):  # Loop over each element in the state vector
        # Compute the sum term for the i-th element according to the formula given in the paper
        sum_term = sum(W[i, j] * (x[j] / (1 + x[j])) for j in range(N) if j != i)  # list comprehension
        derivative_vector[i] = -x[i] + sum_term
    return derivative_vector

In [87]:
# Initialize parameters
N = 100  # Number of genes in a cell
avg_deg = 3
M = 50  # Number of "defects" of the matrix W
W = [np.zeros((N, N)) for _ in range(M)]  # Initialize the weight matrix with zeros; W[defect number, row, column]

initial_conditions = [np.random.rand(N) for _ in range(N)]

dist = lambda p: random.random() < p
q = 1  # Affects the range of the random numbers generated
p = 1 / M  # Probability to change the non-zero elements in w (defect the weight matrix)

num_of_time_stamps = 1000
t_final = 10
t = np.linspace(0, t_final, num_of_time_stamps)

# TODO: Figure out how to make the odeint function stop when the system reaches a steady state or after a certain time period

In [88]:
# Set the base weight matrix
for i in range(N):
    # TODO: this is weird since I defined avg_deg as the average degree but I use it as probability (actual average might be different)
    W[0][i] = [np.random.uniform(0, 2 * q) if (i != j and dist(avg_deg / (N - 1))) else 0 for j in range(N)]

# Create defects of the weight matrix
base_model = W[0]
for m in range(M):
    for i in range(N):
        for j in range(N):
            if base_model[i, j] != 0:
                # Change the value with probability p*(m + 1) (higher probability for higher m) 
                W[m][i, j] = np.random.uniform(0, 2 * q) if dist(p * (m + 1)) else base_model[i, j]
            else:
                W[m][i, j] = 0

In [ ]:
# Initialize the results array
results = [np.zeros((num_of_time_stamps, N)) for _ in range(M)]

# Solve the ODEs for all M weight matrix and corresponding initial conditions
for m in range(M):
    results[m] = odeint(ode_system, initial_conditions[m], t, args=(W[m],))

In [ ]:
# Plot the results
for m in range(M):
    for n in range(N):
        plt.plot(t, results[m][:, n])
plt.xlabel('Time')
plt.ylabel('x value')
plt.title('Michaelis-Menten kinetics')
plt.show()

# ---------

In [ ]:
# initial state
results[0][0, :]

In [ ]:
final_state = [np.random.rand(N) for _ in range(N)]
# final state
for m in range(M):
    final_state[m] = results[m][-1, :]

In [ ]:
# Calculate the variance of the final state
final_state_variance = np.var(final_state)
print(final_state_variance)

In [ ]:
# W2_variance = np.var(W_strong_influence)
# print(W_strong_influence)

In [ ]:
# Save the final state vector to a csv file
# np.savetxt('final_state.csv', final_state, delimiter=',')